<a href="https://colab.research.google.com/github/adamott/AdamTraining/blob/main/Day11_personality_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers gradio --quiet

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
chatbot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

personalities = {
    "Helpful Assistant": "You are a friendly and helpful assistant who provides clear, supportive answers.",
    "Sarcastic Bot": "You respond with dry, witty sarcasm, but you're still informative.",
    "Hawaiian Tour Guide": "You speak with warmth and Aloha spirit, and give responses like a Hawaiian local tour guide."
}

def format_chat_history(history):
    formatted = []
    for user_msg, bot_msg in history:
        formatted.append(f"User: {user_msg}{tokenizer.eos_token}")
        formatted.append(f"Bot: {bot_msg}{tokenizer.eos_token}")
    return "\n".join(formatted)

def chatbot(user_input, history, personality):
    history = history or []
    persona_intro = personalities.get(personality, "")
    conversation = format_chat_history(history)

    prompt = f"{persona_intro}\n\n{conversation}User: {user_input}{tokenizer.eos_token}\nBot:"

    response = chatbot_pipeline(
        prompt,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        truncation=True
    )[0]['generated_text']

    try:
        bot_reply = response.split(prompt)[1].split(f"{tokenizer.eos_token}")[0]
        bot_reply = bot_reply.split("Bot:")[-1].strip()
    except:
        bot_reply = "Oops, I got confused. Try again?"

    history.append((user_input, bot_reply))
    history = history[-5:]

    return "", history

with gr.Blocks() as demo:
    gr.Markdown("# Personalized Chatbot")
    personality = gr.Dropdown(label="Choose a Personality", choices=list(personalities.keys()), value="Helpful Assistant")
    chatbot_interface = gr.Chatbot(height=500)
    user_input = gr.Textbox(label="Say something", placeholder="Talk to your chatbot")
    clear_btn = gr.Button("Clear Chat")

    user_input.submit(chatbot, inputs=[user_input, chatbot_interface, personality], outputs=[user_input, chatbot_interface])
    clear_btn.click(lambda: [], None, chatbot_interface, queue=False)

demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-1-88d597e01400>:55: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(height=500)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c372982b7c1cc39bc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
